# Automating YouTube Script Writing with LangChain and Streamlit

LangChain is a natural language processing capability that generates high-quality, human-like language.
In this project, we will build a youtube script-generating tool using LangChain and Streamlit.

The application’s interface is designed using Streamlit, which provides a user-friendly experience for the creators. 

The application’s prompt templates and chat history storage capabilities make it easy for creators to **customize the generated scripts** to their specific needs. Moreover, the language model generates not only the script but also the **video title**, making the tool a comprehensive solution for YouTube creators.


## Table of Contents:
### 1. Background & Problem Statment
### 2. Setting Up Working Environment
### 3. Building the Application
3.1. Setting the Application Interface
3.2. Setting Prompt Template
3.3. Storing Chat History
3.4. Set up instances of the LLMChain to generate title & script
3.5. Printing the Generated Script
### 4. Running the Application


## 1. Background & Problem Statment

### 1.1. What is LangChain & What it is used for?

LangChain is a framework built around LLMs. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more. The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs. Chains may consist of multiple components from several modules:

- **Prompt templates**: Prompt templates are templates for different types of - prompts. Like “chatbot” style templates, ELI5 question-answering, etc
- **LLMs**: Large language models like GPT-3, BLOOM, etc
- **Agents**: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all are packaged into a logical loop of operations.
- **Memory**: Short-term memory, long-term memory.

### 1.2. Project Overview
In this project, we will build a youtube video writing assistant that will help youtube creators in building their video scripts in less time. You will give the application the topic you would like to create a video on and it will generate a title and the script for the video.

## Setting up Working Environment
Let's first start by creating a new environment called writing_assistant using this command:

After creating the environment you can activate it using the following command:

Finally, you can download all the requirements that will need throughout the project using the following command:

Let's load the libraries and packages we need:

In [ ]:
# Import libraries & packages
import os 
import streamlit as st 
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 

os.environ['OPENAI_API_KEY'] = apikey # your OpenAI api key

## 3. Building the Application 
### 3.1. Setting the Application Interface
First we setup the application layout using the streamlit functions: title, image, and text_input as shown in the code below:

In [ ]:
# Streamlet App framework
st.title('🦜🔗 YouTube Script Assistant') # setting teh title 
st.image('./Youtube.png') # set the featured image of the web application 
prompt = st.text_input('Plug in your prompt here')  # The box for the text prompt

- The first line sets the title of the web application to “🦜🔗 YouTube Script Assistant” using the st.title() function.
- The second line uses the st.image() function to display an image in the web application. It loads the image from the file path "./Youtube.png".
- The third line creates a text input box where users can enter their input. The st.text_input() function takes a string argument that serves as the label for the input box. In this case, the label is "Plug in your prompt here".

### 3.2. Setting Prompt Template
Next, we will set up the prompt template we will make the default template write me a youtube video title about and you will only add the topic that you would like the application to generate the script about:

In [ ]:
# Prompt templates
title_template = PromptTemplate(
    input_variables = ['topic'], 
    template='write me a youtube video title about {topic}'
)

Also, we would like to leverage Wikipedia research for the given title and we can do this using the code below:

In [ ]:
# Prompt templates

script_template = PromptTemplate(
    input_variables = ['title', 'wikipedia_research'], 
    template='write me a youtube video script based on this title TITLE: {title} while leveraging this wikipedia reserch:{wikipedia_research} '
)

### 3.3. Storing Chat History
Next, we will save the history of the results so if you would like to see the previous results use the following code:

In [ ]:
title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')
script_memory = ConversationBufferMemory(input_key='title', memory_key='chat_history')

We first get the title of the memory which is the topic we asked the application to write the script for and then we get the generated script.

### 3.4. Set up instances of the LLMChain to generate title & script
Next, we would like to use langchain to generate the script. First, we will define two instances of the LLMChain class, which is a custom class used to generate text prompts and responses using a language model.

In [ ]:
# Llms
llm = OpenAI(temperature=0.9) 
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True, output_key='title', memory=title_memory)
script_chain = LLMChain(llm=llm, prompt=script_template, verbose=True, output_key='script', memory=script_memory)

wiki = WikipediaAPIWrapper()

- The first line creates an instance of the OpenAI class with temperature=0.9. This sets the temperature of the language model to 0.9, which affects the creativity of the generated text. The instance of OpenAI is stored in the variable llm.

- The second line creates an instance of the LLMChain class with llm=llm, prompt=title_template, verbose=True, and memory=title_memory. This instance is named title_chain and is used to generate a title based on a template. The prompt argument is the prompt template object created earlier that specifies the format of the title. The verbose argument is set to True, which means that information about the text generation process will be printed on the console. The memory argument is set to the title_memory instance, which means that the conversation history for title generation will be stored in that buffer.

- The third line creates another instance of the LLMChain class with llm=llm, prompt=script_template, verbose=True, and memory=script_memory. This instance is named script_chain and is used to generate a script based on a template. Similar to title_chain, the prompt argument is set to the prompt template object for script generation, verbose is set to True, and memory is set to the script_memory instance.

### Printing the Generated Script
Finally, we would like to print the generated script if there is an input prompt. There will be three main generated texts:

- Title: This is a suggested title for the video
- Script: This is the generated script
- Wiki search: This is a Wikipedia research on the given topic

In [ ]:
# Printing the generated script to the screen if there's an input prompt
if prompt: 
    title = title_chain.run(prompt)
    wiki_research = wiki.run(prompt) 
    script = script_chain.run(title=title, wikipedia_research=wiki_research)

    st.write(title) 
    st.write(script) 

    with st.expander('Title History'): 
        st.info(title_memory.buffer)

    with st.expander('Script History'): 
        st.info(script_memory.buffer)

    with st.expander('Wikipedia Research'): 
        st.info(wiki_research)


- The first line checks whether prompt is defined, which means that the user has entered a prompt. If prompt is defined, the application generates a title and a script using the title_chain and script_chain instances of the LLMChain class.

- The title_chain.run(prompt) method generates a title based on the prompt entered by the user, and the wiki.run(prompt) method retrieves Wikipedia research data based on the same prompt. These two outputs are then passed as input to the script_chain.run(title=title, wikipedia_research=wiki_research) method, which generates the script.

- The generated title and script are then displayed on the screen using the st.write() method.

- The next three blocks of code create expanders using st.expander(), which allow the user to view the conversation history for the title, script, and Wikipedia research.

- The conversation history is displayed using the st.info() method.


### 4. Running the application
Now the application is ready to run it. To run the application you can run the following command using the command line in the project directory: